In [12]:
import praw

reddit = praw.Reddit(client_id='xxxxxxxx',
                    client_secret='xxxxxxxxxxxx',
                    user_agent='xxxxxxxxxxxx')

In [13]:
print(reddit.read_only)

True


In [48]:
#參數設定
keywordlist = ['Abort', 'termination of pregnancy', 'terminate pregnancy',
               'pregnancy termination', 'post-abortion', 'postabortion', 
               'roe v wade', 'prolife', 'right-to-life', 'anti-choice', 
               'pro-abortion', 'pro abortion','Dobbs v. Jackson',
               'abortion']
area = 'antimeme' # 'memes' 'dankmemes' politicalmemes antimeme meme

#sort='hot' #手動添加
#time_filter='year' #手動添加

In [ ]:
import os
import praw
import csv
from datetime import datetime
from tqdm import tqdm 

# 選擇一個subreddit
subreddit = reddit.subreddit(area)

# 定義要抓取的帖子數量
#limit = 1000

allmeme = 0

for keyword in keywordlist:
    
    # 設定你想要檢查和創建的資料夾路徑
    folder_path = 'reddit_' + area + '\\' + keyword + '\\'

    # 檢查資料夾是否存在
    if not os.path.exists(folder_path):
        # 如果資料夾不存在，則創建它
        os.makedirs(folder_path)
        print(f"資料夾 '{folder_path}' 已創建。")
    else:
        # 如果資料夾已存在，則輸出提示信息
        print(f"資料夾 '{folder_path}' 已存在。")

    # 打開一個CSV文件以寫入
    with open(folder_path + keyword +'.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # 寫入標題行
        writer.writerow(['Title', 'URL', 'post url','Author', 'Score', 
                         'Subreddit', 'Created UTC', 'Selftext', 
                         'Num Comments', 'Upvote Ratio', 'Is Self','comment',
                         'author_flair_text','clicked','distinguished',
                         'edited','id','is_original_content','link_flair_text',
                         'locked','name','over_18','permalink','saved','spoiler','stickied'])
        
        
        
        # 获取subreddit的帖子列表
        submissions = list(subreddit.search(keyword, limit=None)) #沒設limit=None會只有100則貼文
        
        # 使用tqdm包装submissions，以显示进度条
        for submission in tqdm(submissions, desc='Processing submissions'):
            
            
        # 遍歷subreddit的帖子sort=sort, time_filter=time_filter
        #for submission in subreddit.search(keyword):
            # 构建完整的Reddit帖子URL **取評論需要文章連結 直接用submission.url取到的圖片 .jpg/.png的連結
            post_url = f"https://www.reddit.com{submission.permalink}"
            
            submissioncomment = reddit.submission(url=post_url)
            # 在遍历评论之前，尝试加载所有可用的评论
            submissioncomment.comments.replace_more(limit=None)
            
            commentlist = []

            for comment in submissioncomment.comments.list():
                if isinstance(comment, praw.models.MoreComments):
                    continue  # 如果是MoreComments对象，跳过这个迭代
                
                commentpair = []
                commentpair.append(comment.body)
                commentpair.append(datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'))
                commentlist.append(commentpair)
                
            #把評論轉成json格式
            comments_json = json.dumps(commentlist, ensure_ascii=False)
                
            # 轉換創建時間
            created_time = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            # 寫入每個帖子的資訊
            writer.writerow([submission.title, 
                             submission.url,
                             post_url,
                             str(submission.author), 
                             submission.score,
                             str(submission.subreddit), 
                             created_time, 
                             submission.selftext,
                             submission.num_comments, 
                             submission.upvote_ratio, 
                             submission.is_self,
                             commentlist,
                             submission.author_flair_text, ##
                             submission.clicked,
                             submission.distinguished,
                             submission.edited,
                             submission.id,
                             submission.is_original_content,
                             submission.link_flair_text,
                             submission.locked,
                             submission.name,
                             submission.over_18,
                             submission.permalink,
                             submission.saved,
                             submission.spoiler,
                             submission.stickied])



    import pandas as pd

    # 讀取CSV檔案
    csv_file_path = folder_path + keyword +'.csv'
    df = pd.read_csv(csv_file_path)

    print('貼文數',len(df['URL']))

    import requests
    import time

    error = []
    memenumber = 0

    for i in df['URL']:

        try:

            n = i.split('redd.it/')[1]

            pic = requests.get(i)
            print(pic,i)
            img2 = pic.content #圖片裡的內容
            imgpath = 'reddit_' + area + '\\' + keyword +'\\' + n #存成jpg檔
            pic_out = open(imgpath,'wb') 
            pic_out.write(img2) #將get圖片存入img2
            pic_out.close() #關閉檔案(很重要)
            memenumber += 1

            time.sleep(1)
        except Exception as e:
            error.append(i)

    # 指定CSV文件名
    csv_file_name = 'reddit_' + area + '\\' + keyword +'\\' + 'error.csv'

    with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # 對於一維列表，迭代每個元素，並將每個元素作為一行寫入
        for element in error:
            writer.writerow([element])  # 注意element被放在列表中，因為writerow()期望一個可迭代對象

    print(f"擷取失敗貼文已寫入 {csv_file_name}")
    print('關鍵字',keyword,'迷因數量',memenumber)
    
    allmeme += memenumber
    time.sleep(10)
print('共',allmeme,'個迷因')

In [ ]:
from datetime import datetime

# 獲取「迷因」子版塊的熱門帖子
#subreddit = reddit.subreddit('memes')
abortion = subreddit.search('termination of pregnancy',limit=None)
m = 0

# 遍歷帖子並獲取相關數據
for post in subreddit.hot(limit=None):
    m+=1
    title = post.title
    author = post.author
    created_date = post.created
    created_date = datetime.fromtimestamp(created_date)
    score = post.score
    text = post.selftext
    url = post.url
    num_comments = post.num_comments
    # 其他數據...

    # 打印數據
    print('p88',post)
    print(f"標題: {title}")
    print(f"作者: {author}")
    print(f"創建日期: {created_date}")
    print(f"評分: {score}")
    print(f"text: {text}")
    print(f"評論數: {num_comments}")
    print(f"url: {url}")
    # 其他數據...
    print("=========================")

print(m)

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# 你的Reddit客户端凭据
client_id='xxxxxxx'
client_secret='Nxxxxxxxx'
user_agent='gxxxxxxxx5'

# 准备请求
data = {
    'grant_type': 'client_credentials'
}

# 使用HTTP基本认证
auth = HTTPBasicAuth(client_id, client_secret)

# 设置合适的User-Agent
headers = {
    'User-Agent': user_agent
}

# 发送请求 https://www.reddit.com/api/v1/access_token
response = requests.post('https://www.reddit.com/api/v1/access_token', data=data, auth=auth, headers=headers)

# 解析响应
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print("Access Token:", access_token)
else:
    print("Failed to obtain token, status code:", response.status_code)

In [ ]:
import requests
from datetime import datetime

# 假定已获取的认证令牌
headers = {
    'Authorization': 'xxxx',
}

# API参数
params = {
    'q': 'abortion subreddit:memes',
    'limit': None,
    'sort': 'relevance', # 或 'activity'
    'show_users': 'false',
}

response = requests.get('https://www.reddit.com/subreddits/search.json', headers=headers, params=params)

if response.status_code == 200:
    # 处理响应数据
    memes_about_abortion = response.json()
    # 此处添加你的逻辑来处理或显示结果
    posts = memes_about_abortion['data']['children']

    for item in posts:
        post = item['data']
        title = post['title']
        #author = post['author']
        created_date = post['created']
        # 将UNIX时间戳转换为datetime对象
        created_date = datetime.fromtimestamp(created_date)
        #score = post['score']
        text = post.get('selftext', '')  # 使用get以避免KeyError，如果没有'selftext'则默认为空字符串
        url = post['url']
        #num_comments = post['num_comments']

        # 尝试提取图片链接
        image_url = None
        if 'preview' in post and 'images' in post['preview']:
            image_url = post['preview']['images'][0]['source']['url']
        elif url.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            image_url = url  # 如果URL直接指向一个图片文件
            
        # 打印信息，或者按需处理
        print(f"Title: {title}")
        #print(f"Author: {author}")
        print(f"Created Date: {created_date}")
        #print(f"Score: {score}")
        print(f"Text: {text}")
        print(f"URL: {url}")
        #print(f"Number of Comments: {num_comments}")
        if image_url:
            print(f"Image URL: {image_url}")
        print("------------------------------------------------------")
else:
    print('请求失败:', response.status_code)


API